In [3]:
from imagen_pytorch import Unet, Imagen, SRUnet256, ImagenTrainer
from imagen_pytorch.data import Dataset

In [31]:
unet1 = Unet(
    dim = 32,
    cond_dim = 512,
    dim_mults = (1, 2, 4, 8),
    num_resnet_blocks = 3,
    layer_attns = (False, True, True, True),
)

unet2 = Unet(
    dim = 32,
    cond_dim = 512,
    dim_mults = (1, 2, 4, 8),
    num_resnet_blocks = (2, 4, 8, 8),
    layer_attns = (False, False, False, True),
    layer_cross_attns = (False, False, False, True)
)

# imagen, which contains the unets above (base unet and super resoluting ones)

imagen = Imagen(
    unets = (unet1, unet2),
    text_encoder_name = 't5-large',
    image_sizes = (64, 256),
    timesteps = 1000,
    cond_drop_prob = 0.1
)

# wrap imagen with the trainer class

trainer = ImagenTrainer(imagen)

# mock images (get a lot of this) and text encodings from large T5

text_embeds = torch.randn(64, 256, 1024)
images = torch.randn(64, 3, 256, 256)

# feed images into imagen, training each unet in the cascade

loss = trainer(
    images,
    text_embeds = text_embeds,
    unet_number = 1,            # training on unet number 1 in this example, but you will have to also save checkpoints and then reload and continue training on unet number 2
    max_batch_size = 4          # auto divide the batch of 64 up into batch size of 4 and accumulate gradients, so it all fits in memory
)

# trainer.update(unet_number = 1)

# # do the above for many many many many steps
# # now you can sample an image based on the text embeddings from the cascading ddpm

# images = trainer.sample(texts = [
#     'a puppy looking anxiously at a giant donut on the table',
#     'the milky way galaxy in the style of monet'
# ], cond_scale = 3.)

# images.shape # (2, 3, 256, 256)

In [32]:
trainer = ImagenTrainer(imagen)

In [33]:
trainer.load('checkpoint.pt')

RuntimeError: Error(s) in loading state_dict for Imagen:
	Unexpected key(s) in state_dict: "unets.0.downs.0.1.cross_attn.fn.null_kv", "unets.0.downs.0.1.cross_attn.fn.norm.g", "unets.0.downs.0.1.cross_attn.fn.to_q.weight", "unets.0.downs.0.1.cross_attn.fn.to_kv.weight", "unets.0.downs.0.1.cross_attn.fn.to_out.0.weight", "unets.0.downs.0.1.cross_attn.fn.to_out.1.g", "unets.0.ups.3.0.cross_attn.fn.null_kv", "unets.0.ups.3.0.cross_attn.fn.norm.g", "unets.0.ups.3.0.cross_attn.fn.to_q.weight", "unets.0.ups.3.0.cross_attn.fn.to_kv.weight", "unets.0.ups.3.0.cross_attn.fn.to_out.0.weight", "unets.0.ups.3.0.cross_attn.fn.to_out.1.g". 
	size mismatch for unets.0.text_to_cond.weight: copying a param with shape torch.Size([512, 1024]) from checkpoint, the shape in current model is torch.Size([512, 768]).
	size mismatch for unets.1.text_to_cond.weight: copying a param with shape torch.Size([512, 1024]) from checkpoint, the shape in current model is torch.Size([512, 768]).